In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.


## Read Data 

In [2]:
def read_data(path, include_label):
    def scan_file():
        with open(path) as in_file:
            next(in_file)  # skip header
            for l in in_file:
                yield l.strip().split('\t')
    if include_label:
        return [(qid, did, q, d, float(label)) for qid, q, _, _, did, d, label in scan_file()]
    else:
        return [(qid, did, q, d) for qid, q, _, _, did, d, _ in scan_file()]

train = read_data('data/WikiQA/WikiQACorpus/WikiQA-train.tsv', include_label=True)
predict  = read_data('data/WikiQA/WikiQACorpus/WikiQA-test.tsv', include_label=False)

## Pack Data

In [3]:
train_pack = mz.pack(train[:10000])
predict_pack = mz.pack(predict[:1000])

In [4]:
train_pack.left.head()

,text_left
id_left,
Q1,how are glacier caves formed?
Q2,How are the directions of the velocity and for...
Q5,how did apollo creed die
Q6,how long is the term for federal judges
Q7,how a beretta model 21 pistols magazines works


In [5]:
train_pack.right.head()

,text_right
id_right,
D1-0,A partly submerged glacier cave on Perito More...
D1-1,The ice facade is approximately 60 m high
D1-2,Ice formations in the Titlis glacier cave
D1-3,A glacier cave is a cave formed within the ice...
D1-4,"Glacier caves are often called ice caves , but..."


In [6]:
train_pack.relation.head()

,id_left,id_right,label
0,Q1,D1-0,0.0
1,Q1,D1-1,0.0
2,Q1,D1-2,0.0
3,Q1,D1-3,1.0
4,Q1,D1-4,0.0


In [7]:
train_pack.frame[:].head()

,id_left,text_left,id_right,text_right,label
0,Q1,how are glacier caves formed?,D1-0,A partly submerged glacier cave on Perito More...,0.0
1,Q1,how are glacier caves formed?,D1-1,The ice facade is approximately 60 m high,0.0
2,Q1,how are glacier caves formed?,D1-2,Ice formations in the Titlis glacier cave,0.0
3,Q1,how are glacier caves formed?,D1-3,A glacier cave is a cave formed within the ice...,1.0
4,Q1,how are glacier caves formed?,D1-4,"Glacier caves are often called ice caves , but...",0.0


In [8]:
train_pack.append_text_length(inplace=True)
train_pack.frame[:].head()

Processing length_right with len: 100%|██████████| 9526/9526 [00:00<00:00, 458340.77it/s]


,id_left,text_left,length_left,id_right,text_right,length_right,label
0,Q1,how are glacier caves formed?,29,D1-0,A partly submerged glacier cave on Perito More...,58,0.0
1,Q1,how are glacier caves formed?,29,D1-1,The ice facade is approximately 60 m high,41,0.0
2,Q1,how are glacier caves formed?,29,D1-2,Ice formations in the Titlis glacier cave,41,0.0
3,Q1,how are glacier caves formed?,29,D1-3,A glacier cave is a cave formed within the ice...,61,1.0
4,Q1,how are glacier caves formed?,29,D1-4,"Glacier caves are often called ice caves , but...",128,0.0


In [9]:
predict_pack.frame[:].head()

,id_left,text_left,id_right,text_right
0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0-0,African immigration to the United States refer...
1,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0-1,The term African in the scope of this article ...
2,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0-2,From the Immigration and Nationality Act of 19...
3,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0-3,African immigrants in the United States come f...
4,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0-4,"They include people from different national, l..."


## Preprocess Data

In [10]:
preprocessor = mz.preprocessors.DSSMPreprocessor()
preprocessor.fit(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 993/993 [00:00<00:00, 5248.83it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 9526/9526 [00:03<00:00, 3097.91it/s]
Fitting vocabulary unit.: 100%|██████████| 816312/816312 [00:00<00:00, 2236408.36it/s]


In [11]:
train_pack_processed = preprocessor.transform(train_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 993/993 [00:00<00:00, 2531.83it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 9526/9526 [00:03<00:00, 2390.70it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 108/108 [00:00<00:00, 4943.90it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 992/992 [00:00<00:00, 2133.33it/s]


## Generate Data

In [12]:
train_generator = mz.DataGenerator(train_pack_processed, batch_size=32)

In [13]:
first_batch = train_generator[0]
last_batch = train_generator[-1]
batch_slice = train_generator[2:5]
X, y = first_batch

In [14]:
X

{'id_left': array(['Q885', 'Q642', 'Q1287', 'Q504', 'Q61', 'Q480', 'Q902', 'Q602',
        'Q1205', 'Q402', 'Q255', 'Q360', 'Q261', 'Q661', 'Q953', 'Q238',
        'Q464', 'Q1333', 'Q1257', 'Q1405', 'Q590', 'Q799', 'Q816', 'Q970',
        'Q13', 'Q208', 'Q1143', 'Q247', 'Q654', 'Q483', 'Q654', 'Q466'],
       dtype='<U5'), 'text_left': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), 'length_left': array([38, 44, 25, 32, 24, 26, 38, 41, 19, 34, 44, 25, 28, 52, 38, 32, 48,
        21, 41, 41, 46, 25, 29, 33, 34, 36, 20, 24, 27, 66, 27, 48]), 'id_right': array(['D856-1', 'D627-7', 'D1231-5', 'D497-2', 'D61-4', 'D473-3',
        'D872-6', 'D591-14', 'D1154-13', 'D400-9', 'D255-2', 'D359-13',
        'D261-0', 'D645-0', 'D921-7', 'D238-1', 'D457-16', 'D1274-1',
        'D1202-12', 'D1197-3', 'D

In [15]:
y

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [16]:
X, y = batch_slice
y.shape

(96,)

## Model Building

In [17]:
ranking_task = mz.tasks.Ranking()
ranking_task.metrics = [
    'mae', 'map', 'precision',
    mz.metrics.Precision(k=3),
    mz.metrics.DiscountedCumulativeGain(k=1),
    mz.metrics.DiscountedCumulativeGain(k=3),
    mz.metrics.DiscountedCumulativeGain(k=5),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=1),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5)
]
ranking_task.metrics

['mae',
 mean_average_precision(0),
 precision@1(0),
 precision@3(0),
 discounted_cumulative_gain@1(0),
 discounted_cumulative_gain@3(0),
 discounted_cumulative_gain@5(0),
 normalized_discounted_cumulative_gain@1(0),
 normalized_discounted_cumulative_gain@3(0),
 normalized_discounted_cumulative_gain@5(0)]

In [18]:
model = mz.models.DSSMModel()
input_shapes = preprocessor.context['input_shapes']
model.params['input_shapes'] = input_shapes
model.params['task'] = ranking_task
model.guess_and_fill_missing_params()
model.build()
model.compile()

## Model Training

In [19]:
model.fit(*train_pack_processed.unpack(), epochs=2, batch_size=32)

Epoch 1/2
10000/10000 [==============================] - 11s 1ms/step - loss: 0.0569 - mean_absolute_error: 0.1513
Epoch 2/2
10000/10000 [==============================] - 11s 1ms/step - loss: 0.0380 - mean_absolute_error: 0.1053


In [20]:
model.fit_generator(train_generator, epochs=2, workers=4, use_multiprocessing=True)

Epoch 1/2
313/313 [==============================] - 12s 40ms/step - loss: 0.0299 - mean_absolute_error: 0.1003
Epoch 2/2
313/313 [==============================] - 12s 40ms/step - loss: 0.0246 - mean_absolute_error: 0.0995


### Training with Dynamic Data Generator

In [23]:
pp_no_hashing = mz.preprocessors.DSSMPreprocessor(with_word_hashing=False)
train_pack_processed_no_hashing = pp_no_hashing.fit_transform(train_pack)
word_hashing_unit = mz.processor_units.WordHashingUnit(pp_no_hashing.context['term_index'])
dynamic_data_generator = mz.data_generator.DynamicDataGenerator(word_hashing_unit.transform,
                                                                train_pack_processed_no_hashing)
model.fit_generator(dynamic_data_generator, epochs=2, workers=4, use_multiprocessing=True)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 993/993 [00:00<00:00, 6942.02it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 9526/9526 [00:02<00:00, 3819.83it/s]
Fitting vocabulary unit.: 100%|██████████| 816312/816312 [00:00<00:00, 2844212.93it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 993/993 [00:00<00:00, 8610.01it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 9526/9526 [00:02<00:00, 4128.17it/s]


Epoch 1/2
313/313 [==============================] - 12s 39ms/step - loss: 0.0175 - mean_absolute_error: 0.0914
Epoch 2/2
313/313 [==============================] - 13s 42ms/step - loss: 0.0136 - mean_absolute_error: 0.0840


## Model Predicting 

In [24]:
model.predict(predict_pack_processed[:10].unpack()[0])

array([[ 0.38934442],
       [ 0.23543368],
       [ 0.16310099],
       [ 0.27964133],
       [ 0.0442735 ],
       [ 0.32706514],
       [ 0.23233075],
       [ 0.08352219],
       [ 0.0982188 ],
       [-0.063779  ]], dtype=float32)

## Model Evaluating

In [25]:
model.evaluate(*train_pack_processed.unpack())

10000/10000 [==============================] - 2s 160us/step


{'loss': 0.011673345744609832,
 'mean_absolute_error': 0.07965075917243958,
 'mean_average_precision(0)': 0.41515609264853975,
 'precision@1(0)': 0.4149043303121853,
 'precision@3(0)': 0.1624706277274253,
 'discounted_cumulative_gain@1(0)': 0.5985804197847461,
 'discounted_cumulative_gain@3(0)': 0.6619165128669006,
 'discounted_cumulative_gain@5(0)': 0.6667312223859859,
 'normalized_discounted_cumulative_gain@1(0)': 0.4149043303121853,
 'normalized_discounted_cumulative_gain@3(0)': 0.4149043303121853,
 'normalized_discounted_cumulative_gain@5(0)': 0.41533804285807996}